In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('default') # set style (colors, background, size, gridlines etc.) # ggplot
plt.rcParams['figure.figsize'] = 8, 4 # set default size of plots
plt.rcParams.update({'font.size': 18})

import scraping_class
logfile = 'log.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [2]:
#pip install cpi

In [3]:
movies = pd.read_csv('IMDb movies.csv')
actors = pd.read_csv('IMDb names.csv')
ratings = pd.read_csv('IMDb ratings.csv')
role = pd.read_csv('IMDb title_principals.csv')

##### IMDb - Remove duplicates and all years before 1990

In [4]:
movies_90 = movies[movies.year >= 1990] # (53368, 22)
#movies_90 = movies.drop(movies[movies['year'] < 1990].index)
movies_us = movies[movies.country == 'USA']
movies_us_90 = movies_us[movies_us.year >= 1990]
movies_90['title'] = movies_90['title'].astype(str)
movies_90 = movies_90.drop_duplicates(subset = 'title')
mocies_us_90 = movies_us_90.drop_duplicates(subset = 'title')

/Users/ea/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


##### RT - Remove duplicates 

In [5]:

df_RT = pd.read_csv('RT.csv').drop_duplicates(subset = 'title')
df_90 = pd.merge(movies_90,df_RT,how='inner',on='title')
df_90 = df_90.drop_duplicates()
df_90_us = pd.merge(movies_us_90,df_RT,how='inner',on='title')
df_90_us = df_90_us.drop_duplicates()

In [6]:
# **** Remove NaN ****
df_90_income = df_90.dropna(subset = ['worlwide_gross_income']).reset_index(drop=True).drop_duplicates()
df_us = df_90_us[df_90_us['worlwide_gross_income'].notna()].reset_index(drop=True)

#df_90_income.dtypes
#df_90_income.head()
#print(df_90_income.shape)
#df_90
#df_90_income

##### Cleaning dataset and adjusting income and budget for inflation (CPI)

In [7]:
import cpi
#cpi.update()

def inflate_column(data, column):
    '''Inflate monetary figures from a given year to their 2018 values'''
    return data.apply(lambda x: cpi.inflate(x[column], x.year), axis=1)

In [8]:
df_90_income['worldwide_gross_income'] = df_90_income['worlwide_gross_income']\
                                        .apply(lambda x:x[1:] if x and x[:1]== '$' else None)
df_90_income = df_90_income.drop('worlwide_gross_income', axis=1)
df_90_income = df_90_income.dropna(subset = ['worldwide_gross_income'])

In [9]:
df_90_income = df_90_income.dropna(subset = ['budget'])
df_90_income['budget'] = df_90_income['budget'].apply(lambda x:x[1:] if x and x[:1]== '$' else None)

df_90_income = df_90_income.dropna(subset = ['usa_gross_income'])
df_90_income['usa_gross_income'] = df_90_income['usa_gross_income']\
                                        .apply(lambda x:x[1:] if x and x[:1]== '$' else None)

In [10]:
df_90_income = df_90_income.reset_index(drop=True).drop_duplicates()

df_90_income.worldwide_gross_income = df_90_income.worldwide_gross_income.astype(int)
df_90_income = df_90_income[df_90_income.year <= 2018]
df_90_income['adjusted_wgi'] = inflate_column(df_90_income, 'worldwide_gross_income')

df_budget = df_90_income.dropna(subset = ['budget']).copy()
df_budget.budget = df_budget.budget.astype(int)
df_budget['adjusted_budget'] = inflate_column(df_budget, 'budget')

df_budget = df_budget.dropna(subset = ['usa_gross_income']).copy()
df_budget['usa_gross_income'] = df_budget['usa_gross_income'].astype(int)
df_budget['adjusted_us_income'] = inflate_column(df_budget, 'usa_gross_income')

df_budget['net_income'] = df_budget.adjusted_wgi - df_budget.adjusted_budget
df_budget['net_us_income'] = df_budget['adjusted_us_income'] - df_budget['adjusted_budget']
df = df_budget.reset_index(drop=True)
#df_90_income.dtypes

In [11]:
directors = list(df['director'].values)


In [12]:
df.dtypes

imdb_title_id              object
title                      object
original_title             object
year                        int64
date_published             object
genre                      object
duration                    int64
country                    object
language                   object
director                   object
writer                     object
production_company         object
actors                     object
description                object
avg_vote                  float64
votes                       int64
budget                      int64
usa_gross_income            int64
metascore                 float64
reviews_from_users        float64
reviews_from_critics      float64
tomatometer_score          object
audience_score             object
worldwide_gross_income      int64
adjusted_wgi              float64
adjusted_budget           float64
adjusted_us_income        float64
net_income                float64
net_us_income             float64
dtype: object

In [13]:
# Converting Rotten Tomatoes scores to floats
df['tomatometer_score'] = df['tomatometer_score'].replace({'\%':''}, regex = True)
df['audience_score'] = df['audience_score'].replace({'\%':''}, regex = True)

df['tomatometer_score']= df['tomatometer_score'].astype(float)
df['audience_score']= df['audience_score'].astype(float)

In [17]:
director_scores = []
a = {}
for director in tqdm(directors):
    a[director] = pd.DataFrame(df[df['director'].str.contains(director)])
    director_scores.append([director,
                           a[director]['avg_vote'].mean(),
                           a[director]['metascore'].mean(),
                           a[director]['tomatometer_score'].mean(),
                           a[director]['audience_score'].mean()])

In [18]:
cols = ['director', 'director_imdb', 'director_metascore', 'director_tomatometer', 'director_audience']
df_director = pd.DataFrame(director_scores, index=range(len(director_scores)),columns=cols)

In [19]:
len(directors)

4354

In [20]:
#writers = list(df.dropna(subset = ['writer'])['writer'].values)
#writers = list(df['writer'].values)
df = df.dropna(subset = ['writer'])

In [21]:
writers = list(df['writer'].values)
writer_scores = []
a = {}
for writer in tqdm(writers):
    
    a[writer] = pd.DataFrame(df[df['writer'].str.contains(writer)])
    writer_scores.append([writer,
                           a[writer]['avg_vote'].mean(),
                           a[writer]['metascore'].mean(),
                           a[writer]['tomatometer_score'].mean(),
                           a[writer]['audience_score'].mean()])



In [22]:
cols = ['writer', 'writer_imdb', 'writer_metascore', 'writer_tomatometer', 'writer_audience']
df_writer = pd.DataFrame(writer_scores, index=range(len(writer_scores)),columns=cols)

In [23]:
actors = []

for actor in df['actors']:
    actors.append(actor.split(", ")[:4])

In [30]:
actor_scores = []
a = {}
for film in tqdm(actors):
    for actor in film:
        a[actor] = pd.DataFrame(df[df['actors'].str.contains(actor)])
        actor_scores.append([actor,
                           a[actor]['avg_vote'].mean(),
                           a[actor]['metascore'].mean(),
                           a[actor]['tomatometer_score'].mean(),
                           a[actor]['audience_score'].mean()])

In [39]:
cols = ['actor', 'actor_imdb', 'actor_metascore', 'actor_tomatometer', 'actor_audience']
df_actors = pd.DataFrame(actor_scores, index=range(len(actor_scores)),columns=cols)
#actors[df_actors['actor']=='Meg Ryan']

In [48]:
df = pd.merge(df, df_director, how='left', on='director').drop_duplicates().reset_index(drop=True)
df = pd.merge(df, df_writer, how='left', on='writer').drop_duplicates().reset_index(drop=True)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,director_tomatometer_y,director_audience_y,director_imdb,director_metascore,director_tomatometer,director_audience,writer_imdb,writer_metascore,writer_tomatometer,writer_audience
0,tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-04-05,"Comedy, Fantasy, Romance",118,USA,"English, French",James Mangold,...,69.0,76.142857,7.157143,61.714286,69.0,76.142857,6.400000,44.0,50.000000,62.0
1,tt0118887,Cop Land,Cop Land,1997,1997-12-05,"Crime, Drama, Thriller",105,USA,English,James Mangold,...,69.0,76.142857,7.157143,61.714286,69.0,76.142857,6.866667,53.0,59.666667,71.0
2,tt0172493,"Girl, Interrupted","Girl, Interrupted",1999,2000-03-24,"Biography, Drama",127,"USA, Germany",English,James Mangold,...,69.0,76.142857,7.157143,61.714286,69.0,76.142857,7.300000,51.0,54.000000,84.0
3,tt0309698,Identity,Identity,2003,2003-06-13,"Mystery, Thriller",90,USA,English,James Mangold,...,69.0,76.142857,7.157143,61.714286,69.0,76.142857,7.300000,64.0,62.000000,75.0
4,tt0358273,Walk the Line,Walk the Line,2005,2006-02-03,"Biography, Drama, Music",136,"USA, Germany","English, Russian",James Mangold,...,69.0,76.142857,7.157143,61.714286,69.0,76.142857,7.800000,72.0,82.000000,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,tt7388562,"Paul, Apostle of Christ","Paul, Apostle of Christ",2018,2018-03-23,"Adventure, Biography, Drama",108,USA,English,Andrew Hyatt,...,45.0,81.000000,6.600000,52.000000,45.0,81.000000,6.600000,52.0,45.000000,81.0
4349,tt7424200,Teen Titans Go! To the Movies,Teen Titans Go! To the Movies,2018,2018-08-03,"Animation, Action, Comedy",84,USA,English,"Aaron Horvath, Peter Rida Michail",...,91.0,72.000000,6.800000,69.000000,91.0,72.000000,6.800000,69.0,91.000000,72.0
4350,tt7668870,Searching,Searching,2018,2018-08-31,"Drama, Mystery, Thriller",102,"Russia, USA",English,Aneesh Chaganty,...,92.0,87.000000,7.700000,71.000000,92.0,87.000000,7.700000,71.0,92.000000,87.0
4351,tt7690670,Superfly,Superfly,2018,2018-09-14,"Action, Crime, Thriller",116,USA,"English, Spanish, Cantonese",Director X.,...,50.0,46.000000,5.100000,52.000000,50.0,46.000000,6.350000,54.0,57.500000,58.5


##### Clean US dataset for valuta (dollar sign)

In [ ]:
df_us['wgi'] = df_us['worlwide_gross_income'].replace({'\$':''}, regex = True)
df_us['budget'] = df_us['budget'].replace({'\$':''}, regex = True)
df_us['usa_gross_income'] = df_us['usa_gross_income'].replace({'\$':''}, regex = True)


In [ ]:
#len(df_us[df_us.year==2019])
df_us.wgi = df_us.wgi.astype(int)
df_us = df_us[df_us.year <= 2018]
df_us['adjusted'] = inflate_column(df_us, 'wgi')
#df_us

In [66]:
df.columns.values

array(['imdb_title_id', 'title', 'original_title', 'year',
       'date_published', 'genre', 'duration', 'country', 'language',
       'director', 'writer', 'production_company', 'actors',
       'description', 'avg_vote', 'votes', 'budget', 'usa_gross_income',
       'metascore', 'reviews_from_users', 'reviews_from_critics',
       'tomatometer_score', 'audience_score', 'worldwide_gross_income',
       'adjusted_wgi', 'adjusted_budget', 'adjusted_us_income',
       'net_income', 'net_us_income', 'director_imdb_x',
       'director_metascore_x', 'director_tomatometer_x',
       'director_audience_x', 'director_imdb_y', 'director_metascore_y',
       'director_tomatometer_y', 'director_audience_y', 'director_imdb_x',
       'director_metascore_x', 'director_tomatometer_x',
       'director_audience_x', 'director_imdb_y', 'director_metascore_y',
       'director_tomatometer_y', 'director_audience_y', 'director_imdb_x',
       'director_metascore_x', 'director_tomatometer_x',
       'dire

##### Seperating data into features (have greater flexibility for project when predicting)

In [67]:
df_ratings = [['avg_vote', 'metascore','tomatometer_score', 'audience_score']]
df_x = df[['title', 'year', 'date_published', 'genre', 'duration','country', 'director', 'actors', 'budget','production_company','writer', 'director_imdb','director_tomatometer','director_audience','writer_imdb','writer_metascore','writer_tomatometer','writer_audience']]
df_y = df_budget[['net_income','net_us_income','adjusted_wgi', 'adjusted_us_income']]

In [68]:
df_x.to_csv('inputInfo_X.csv', encoding='utf-8',index=False)

In [69]:
df_y.to_csv('inputInfo_Y.csv', encoding='utf-8',index=False)

In [70]:
x = pd.read_csv("inputInfo_X.csv")

In [71]:
y = pd.read_csv("inputInfo_Y.csv")

In [72]:
def convert_to_array(rating):
    rating = str(rating)
    rating = rating.lower()
    rating = rating.split(",")
    return rating

##### Converting columns with multiple string entries to a list. This will assist for converting each string entry to dummy variables

In [73]:
x["genre"]= x["genre"].apply(lambda x: convert_to_array(x))
x["country"] = x["country"].apply(lambda x: convert_to_array(x))
x["director"] = x["director"].apply(lambda x: convert_to_array(x))
x["actors"] = x["actors"].apply(lambda x: convert_to_array(x))
x["title"] = x["title"].apply(lambda x: convert_to_array(x))
x["production_company"] = x["production_company"].apply(lambda x: convert_to_array(x))
x["writer"] = x["writer"].apply(lambda x: convert_to_array(x))

##### Converting published date to months

In [74]:
x["date_published"] = x["date_published"].apply(lambda x: pd.to_datetime(x).month)

##### Creating dummy variables 

In [75]:
from sklearn.preprocessing import MultiLabelBinarizer
MLB = MultiLabelBinarizer()

In [76]:
actor_dummy = pd.DataFrame(MLB.fit_transform(x["actors"]),columns = MLB.classes_,index=x.index)
genre_dummy = pd.DataFrame(MLB.fit_transform(x["genre"]),columns=MLB.classes_, index=x.index)
country_dummy = pd.DataFrame(MLB.fit_transform(x["country"]),columns = MLB.classes_,index=x.index)
director_dummy = pd.DataFrame(MLB.fit_transform(x["director"]),columns = MLB.classes_,index=x.index)
production_dummy = pd.DataFrame(MLB.fit_transform(x["production_company"]),columns = MLB.classes_,index=x.index)

In [ ]:
genre_dummy.head()

##### Looking at the predictor values

In [ ]:
y_2 = y[["tomatometer_score","audience_score","metascore","avg_vote","usa_gross_income","worldwide_gross_income"]]

##### Converting the audience and user rating to binary. 

In [ ]:
#y_2["t_rating"] = y_2["tomatometer_score"].apply(lambda x: 1 if x > 60.0 else 0)
#y_2["audience_score"] = y_2["audience_score"].apply(lambda x: 1 if x > 60.0 else 0) 

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold

##### Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))
sns.distplot(df_90_income.year, bins = 20);
plt.title("Distribution of movies over the years", size = 10);

In [ ]:
sns.set(style="white")

plt.figure(figsize=(10,5))
plt.title('Movies by the year', size=15)
sns.distplot(df_90_income.year, kde=False)
plt.ylabel('Number of movies', size=15)
plt.xlabel('Year of release',size=15)
plt.axis([1990, 2019, 0, 700])
plt.xticks(np.arange(1990, 2019, step=2))
plt.show()

In [ ]:
# Overview of the different genres 
df_90_income['first_genre'] = df_90_income['genre'].str.split(',').str[0]

a = plt.cm.binary

plt.figure(figsize=(10,4))
count = df_90_income['first_genre'].value_counts()[:7]
sns.barplot(count.values, count.index, palette=[a(0.1),a(0.2),a(0.3),a(0.4),a(0.5),a(0.6),a(0.7)])
for i, v in enumerate(count.values):
    plt.text(0.8,i,v,color='k',fontsize=10)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Genre name', fontsize=12)
plt.title("Genres", fontsize=15)
plt.show()

In [ ]:
# Overview of movies with highest scores 
top_scored = df_90_income.sort_values(["tomatometer_score","audience_score","metascore","avg_vote"], ascending=False)[
    ["title", "director", "tomatometer_score","audience_score","metascore","avg_vote"]]
top_scored.index = range(len(df_90_income))
top_scored.head(n=10)

In [ ]:
test = df_90_income.sort_values(["year"], ascending=False)[
    ["year","tomatometer_score","audience_score","metascore","avg_vote"]]
test.index = range(len(df_90_income))
#test.head(n=10)


In [ ]:
test.plot(kind="scatter",
                      x="year",
                      y="metascore",
                      alpha=0.4)

plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize = (9,5))
#sns.distplot(df_90_income.tomatometer_score,bins = 50);
#plt.title("Distribution of metascore among movies", size = 10);


# Error: ould not convert string to float: '50%'
#df_90_income = df_90_income[(df_90_income.tomatometer_score.notnull()) &
#                      (df_90_income.audience_score.notnull())]
#sns.jointplot(x=df_90_income['year'], y=df_90_income['tomatometer_score'],
#              kind="kde").fig.set_size_inches(15,15)

# Works but not pretty 
#plt.figure(figsize=[30,10])
#sns.barplot(x=df_90_income.year, y=df_90_income.metascore, data=df_90_income)

In [ ]:
# Ideas:
#1) Look at gender or race in director (men vs women) (balck vs white)

In [ ]:
df_budget.plot(kind="scatter",
                      x="avg_vote",
                      y="net_income",
                      alpha=0.4)